# Обход графа

Граф - это структура, состоящая из вершин (узлов), некоторые из которых соеденены между собой ребрами. С помощью графов можно удобно представлять очень много вещей из реального мира: различные процессы, дороги, социальные взаимосвязи и многое другое. 

По этой причине было бы очень неплохо уметь работать с графами, проводить анализ его структуры, находить необходимую информацию.

<img src="img/graph.png" style="height: 300px;">

Автор: <a href="//commons.wikimedia.org/wiki/User:AzaToth" title="User:AzaToth">User:AzaToth</a> - <a href="//commons.wikimedia.org/wiki/File:6n-graf.png" title="File:6n-graf.png">Image:6n-graf.png</a> simlar input data, Общественное достояние, <a href="https://commons.wikimedia.org/w/index.php?curid=820489">Ссылка</a>

Существует два классических способа хранения графа
* Множество вершин, каждая из которых хранит список вершин, с которыми она соединена
* Матрица (таблица) с 0 и 1, которая представляется следующим образом: если в i, j клетке стоит 1, то значит, i-тая вершина соединена с j-той, если же 0 - не соеденина

Различные способы представления графов полезны в различных ситуациях и выбирать какой-то конкретный необходимо исходя их задачи, которую необходимо решить.

Для наших целей сегодня больше отлично подойдет первый вариант. Реализуем вершину с соседями, а после соберем их в один граф.

In [1]:
#include <fstream>
#include <vector>
#include <iostream>
#include <algorithm>
#include <queue> 

In [2]:
void print_file(std::string f) { // печать файла для наглядности
    std::ifstream file(f);
    std::string s;
    while(std::getline(file, s)) {
        std::cout << s << std::endl;
    }
}

In [3]:
class Vertex { // класс вершины
public:
    Vertex(int i) : id(i), value(0) {}
    ~Vertex(){}

    void connectOneside(Vertex * vertex) {
        neighbours.push_back(vertex);
    }

    void connectTwoside(Vertex * vertex) {
        neighbours.push_back(vertex);
        vertex->connectOneside(this);
    }

    int id;
    int value; // ячейка для хранения какого-то значения в вершине
    std::vector<Vertex*> neighbours; // вершины, с которыми связана эта вершина
};

class Graph { // класс графа
public:
    Graph() {}
    Graph(int size) { // создать граф размена size
        vertexes.reserve(size);
        for(int i = 0; i < size; i++) {
            vertexes.push_back(new Vertex(i));
        }
    }
    
    Graph(std::string path) {
        readFromFile(path);
    }
    ~Graph(){
        for(Vertex * v : vertexes) delete v;
    }

    Vertex * at(int index) { // получить вершину по id
        return vertexes.at(index);
    }

    void readFromFile(std::string path) { // Формат: 2 -> 3 или 2 <-> 3. Это означает, что 2 вершина соединена с 3
        std::ifstream file(path);
        
        int size;
        file >> size;
        
        vertexes.reserve(size);
        for(int i = 0; i < size; i++) {
            vertexes.push_back(new Vertex(i+1));
        }

        int id1, id2;
        std::string type;
        while(file >> id1 >> type >> id2) {
            if(type == "->") vertexes[id1-1]->connectOneside(vertexes[id2-1]);
            else if(type == "<->") vertexes[id1-1]->connectTwoside(vertexes[id2-1]);
        }
    }

    std::vector<Vertex*> vertexes; // вершины графа
};

In [4]:
print_file("files/1.graph");
std::cout << "-------" << std::endl;

Graph g1("files/1.graph");

6
1 <-> 2
1 <-> 5
2 <-> 5
2 <-> 3
4 <-> 5
4 <-> 3
4 <-> 6
-------


(Graph &) @0x7f80f86b4010


In [5]:
for(auto v : g1.vertexes) {
    std::cout << v->id << " -> ";
    for(auto w : v->neighbours) {
        std::cout << w->id << " ";
    }
    std::cout << std::endl;
}

1 -> 2 5 
2 -> 1 5 3 
3 -> 2 4 
4 -> 5 3 6 
5 -> 1 2 4 
6 -> 4 


Как можно видеть, данный граф описывает тот же граф, что изображен на картинке в начале лекции. 

# Поиск в глубину

Для анализа графа чаще всего необходимо пройтись по каждой вершине этого графа и после этого на основе полученной информации сделать какой-то вывод или посчитать какую-то величену. Самым простых алгоритмом обхода графа является поиск в глубину. Его суть заключается в следующем:

* Начинаем с какой-то вершины
* Идем сначала в первого соседа этой вершины
* Повтораяем это действие пока не упремся в вершину, у которой нет еще не просмотренных соседей.
* Возвращемся на одну вершину выше и пытаемся пойти во вторую соседскую вершину таким же способом
* Когда закончатся и эти соседы, то вернемся к вершине выше
* Будем так делать пока не вернемся в исходную вершину и не пройдем всех ее соседей

По сути мы пытаемся уйти сразу на максимальное расстояние от исходной вершины

<img src="img/depth-tree.png">

Автор: <a href="//commons.wikimedia.org/wiki/User:Alexander_Drichel" title="User:Alexander Drichel">Alexander Drichel</a> - <span class="int-own-work" lang="ru">собственная работа</span>, <a href="http://creativecommons.org/licenses/by-sa/3.0" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=3791979">Ссылка</a>

На данной картике изображена последовательность вершин, которую мы пройдем, начиная поиск в грубину из вершины 1.

## Поиск компонент связности графа

Одно из свойств графа - связность. Оно означает, что из любой вершины, графа можно попасть в любую другую вершину графа, если двигаться по ребрам. Если граф не связен, то это означает, что он разбивается на несколько компонент связности - подграфы, которые в совокупности являются исходным графом и при этом каждый из них является связным графом.

Например, граф на рисунке выше - связный, а граф на рисунке ниже - нет и имеет три компоненты связности.

<img src="img/unconn.png">

Автор: Dmitry Dzhus - <span class="int-own-work" lang="ru">собственная работа</span>, Общественное достояние, <a href="https://commons.wikimedia.org/w/index.php?curid=2532274">Ссылка</a>

Для того, чтобы найти все компоненты связности, можно воспользоваться поиском в глубину. Суть алгоритма в следующем:

* Запускаем поиск в глубину у какой-то вершины, помечая каждую вершину, которую найдем, номером компоненты связности (изначально - 1)
* После этого будем искать вершину в графе, которая не помечена никакой компонентой и запустим от нее такой же поиск в грубину, однако теперь будем помечать все вершину уже номером следующей компоненты
* Повторяем этот процесс до тех пор, пока не останется вершин, которые мы бы не пометили
* После этого номер последней компоненты связности - это и есть количество компонент связности в графе, а также каждая вершина будет помечена номером своей компоненты

Обоснование корректности данного алгоритма тривиально.

In [6]:
void dfs_connected(Vertex * v, int index) {
    if(v->value > 0) return; // если уже помечена
    v->value = index; // помечаем
    for(Vertex * w : v->neighbours) dfs_connected(w, index); // проходим по соседям
}

In [7]:
int connected_components(Graph &g) {
    int index = 1; // номер первой компоненты связности
    for(Vertex * v : g.vertexes) {
        if(v->value > 0) continue; // не трогаем вершины, которые уже помечены
        dfs_connected(v, index); // ищем из той вершины, которая еще не помечена
        index++; // выставляем номер следующей компоненты
    }
    return index - 1;
}

In [8]:
print_file("files/2.graph");
std::cout << "--------" << std::endl;

Graph g2("files/2.graph");

print_file("files/3.graph");
std::cout << "--------" << std::endl;

Graph g3("files/3.graph");

12
1 <-> 2
1 <-> 7
1 <-> 8
2 <-> 3
2 <-> 6
8 <-> 9
8 <-> 12
3 <-> 4
3 <-> 5
9 <-> 10
9 <-> 11
--------
7
1 <-> 2
2 <-> 3
3 <-> 4
6 <-> 7
--------


(Graph &) @0x7f80f86b4060


Как можно видеть, граф g2 - это граф на второй картинке, а граф g3 - граф на третьей картинке.

In [9]:
std::cout << connected_components(g2) << std::endl;

1


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f80f410e6e0


In [10]:
std::cout << connected_components(g3) << std::endl;

3


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f80f410e6e0


Результаты совпали с ожидаемыми.

# Поиск в ширину

Вторым стандартным способом является поиск в ширину. В отличии от поиска в грубину, теперь мы не "Убегаем" от начальной вершины, а двигаемся равномерно во все стороны от начальной вершины.

Более формально:
* Вначале мы проходим всех соседей начальной вершины
* Потом проходим всех соседей, которые есть у тех, что мы только что прошли
* Продолжаем этот процесс, пока не пройдем все вершины

Картинка ниже иллюстрирует порядок прохода вершин при поиске в ширину

<img src="img/bfsa.gif">

Автор: <a href="//commons.wikimedia.org/w/index.php?title=User:Mre&amp;action=edit&amp;redlink=1" class="new" title="User:Mre (page does not exist)">Mre</a> - <span class="int-own-work" lang="ru">собственная работа</span>, <a href="http://creativecommons.org/licenses/by-sa/3.0" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=6342695">Ссылка</a>

## Проверка графа на двудольность 

Еще одно свойство графа - двудольность. Оно означает, что все вершины графа можно разделить на два множества (две доли) таки образом, чтобы любая вершина из одной доли была связана только с вершинами из другой доли. 



<img src="img/sbg.png">

Автор: MistWiz - <span class="int-own-work" lang="ru">собственная работа</span>, Общественное достояние, <a href="https://commons.wikimedia.org/w/index.php?curid=1814874">Ссылка</a>

На рисунке выше изображен двудольный граф с долями U и V.

Четвертый граф (который изображен выше этого графа), кстати, тоже двудольный с долями { 1, 5, 6, 7, 8 } и { 2, 3, 4, 9, 10}

Для проверки графа на двудольность можно использовать обход в ширину:

* Начинаем с любой вершины. Обозначим, что она в доле +1.
* Пойдем по всем ее соседям
 * Если текущий сосед еще не помечен, то помечаем его другой долей и добавляем в очередь к просмотру
 * Если он помечен этой же долей, то это означает, что есть связь, когда связаны две вершины из одной доли -> граф не двудольный
 * Если он помечен другой долей, то значит мы уже проверели эту вершину и просто пропускаем ее
 
Из посмотрения алгоритма видно, что при проходе в ширину двудольного графа, из одной вершины мы всегда доджны попадать только в вершины из другой доли. Если это в какой-то момент нарушилось, значит граф не двудольный, если же нет - то двудольный.

In [11]:
bool bipartite_graph(Graph &g) {
    std::queue<Vertex *> q; // очередь вершин, которые нужно пройти
    q.push(g.at(0));
    q.front()->value = 1; // начальная вершина
    while(!q.empty()) {
        Vertex * v = q.front(); // берем следующую вершину
        q.pop();
        
        for(Vertex * w : v->neighbours) {
            if(w->value == 0) {
                w->value = v->value * (-1); // помечаем вершину
                q.push(w); // добавляем в очередь к просмотру
            } else if(w->value == v->value) return false; // нашли связь из одной доли
            // иначе ничего не делаем с вершиной
        }
    }
    return true;
}

In [12]:
print_file("files/2.graph");
std::cout << "--------" << std::endl;

Graph g2_3("files/2.graph");

print_file("files/4.graph");
std::cout << "--------" << std::endl;

Graph g4("files/4.graph");

print_file("files/5.graph");
std::cout << "--------" << std::endl;

Graph g5("files/5.graph");

12
1 <-> 2
1 <-> 7
1 <-> 8
2 <-> 3
2 <-> 6
8 <-> 9
8 <-> 12
3 <-> 4
3 <-> 5
9 <-> 10
9 <-> 11
--------
9
1 <-> 6
2 <-> 6
2 <-> 7
3 <-> 8
3 <-> 9
4 <-> 7
5 <-> 6
5 <-> 9
--------
4
1 <-> 2
2 <-> 3
3 <-> 4
4 <-> 1
1 <-> 3
2 <-> 4
--------


(Graph &) @0x7f80f86b40c0


In [13]:
std::cout << (bipartite_graph(g2_3) ? "Yes" : "No") << std::endl;

Yes


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f80f410e6e0


In [14]:
std::cout << (bipartite_graph(g4) ? "Yes" : "No") << std::endl;

Yes


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f80f410e6e0


In [15]:
std::cout << (bipartite_graph(g5) ? "Yes" : "No") << std::endl;

No


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f80f410e6e0


g2_3 - Граф, изображенный вторым в этой лекции.

g4 - последний изображенный граф

g5 - полный граф на 4 вершинах (изображение ниже)

<img src="img/sg.png">

Автор: <a href="//commons.wikimedia.org/wiki/User:Koko90" title="User:Koko90">Koko90</a> - <span class="int-own-work" lang="ru" xml:lang="ru">собственная работа</span>, <a href="http://creativecommons.org/licenses/by-sa/3.0" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=10947368">Ссылка</a>

Как можно видеть, все результаты совпали с ожидаемыми.